In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Hyperparameters
DIMENSIONS = 10000  # Dimensionality of hypervectors
NUM_CLASSES = 10    # Number of classes in Fashion MNIST
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.01

# Data Preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# HDC Model with Learnable Parameters
class LearnableHDC(nn.Module):
    def __init__(self, input_size, num_classes, dimensions):
        super(LearnableHDC, self).__init__()
        self.dimensions = dimensions
        self.num_classes = num_classes
        self.input_size = input_size

        # Learnable projection matrix
        self.projection = nn.Parameter(torch.randn(input_size, dimensions))
        # Learnable class hypervectors
        self.class_vectors = nn.Parameter(torch.randn(num_classes, dimensions))

    def forward(self, x):
        batch_size = x.size(0)
        x = x.view(batch_size, -1)  # Flatten images
        proj = torch.matmul(x, self.projection)  # Project to high-dimensional space
        proj = torch.tanh(proj)  # Non-linearity

        # Compute similarity with class hypervectors
        similarities = torch.matmul(proj, self.class_vectors.t())
        return similarities

# Initialize model, loss, and optimizer
model = LearnableHDC(28 * 28, NUM_CLASSES, DIMENSIONS)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training Loop
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {running_loss / len(train_loader)}")

# Testing Loop
def test_model():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

test_model()

100%|██████████| 26.4M/26.4M [00:01<00:00, 17.5MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 309kB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:00<00:00, 5.52MB/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 12.5MB/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Epoch 1/10, Loss: 17.196963158751856
Epoch 2/10, Loss: 9.315310281476995
Epoch 3/10, Loss: 8.038295135315039
Epoch 4/10, Loss: 7.455595641248007
Epoch 5/10, Loss: 7.422228387678102
Epoch 6/10, Loss: 6.8049886272406015
Epoch 7/10, Loss: 7.194910476456827
Epoch 8/10, Loss: 6.645380228567225
Epoch 9/10, Loss: 6.587520457534139
Epoch 10/10, Loss: 5.950123872838295
Test Accuracy: 73.63%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

# Hyperparameters
DIMENSIONS = 10000
NUM_CLASSES = 10
BATCH_SIZE = 128
EPOCHS = 50 # Increase epochs
LEARNING_RATE = 0.001 # Try lower learning rate
WEIGHT_DECAY = 1e-5

# Data Preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# HDC Model with Learnable Parameters
class LearnableHDC(nn.Module):
    def __init__(self, input_size, num_classes, dimensions):
        super(LearnableHDC, self).__init__()
        self.dimensions = dimensions
        self.num_classes = num_classes
        self.input_size = input_size

        # Using a fixed random projection (can experiment with learnable too)
        self.projection = torch.randn(input_size, dimensions)
        # Learnable class hypervectors
        self.class_vectors = nn.Parameter(torch.randn(num_classes, dimensions))

    def forward(self, x):
        batch_size = x.size(0)
        x = x.view(batch_size, -1)
        # Ensure projection matrix is on the same device as input
        self.projection = self.projection.to(x.device)
        proj = torch.matmul(x, self.projection)
        proj = torch.tanh(proj)

        # Compute cosine similarity
        similarities = F.cosine_similarity(proj.unsqueeze(1), self.class_vectors.unsqueeze(0), dim=2)
        return similarities

# Initialize model, loss, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LearnableHDC(28 * 28, NUM_CLASSES, DIMENSIONS).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Training Loop
best_accuracy = 0
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device) # Move data to GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {running_loss / len(train_loader)}")

    # Testing and saving best model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), "best_hdc_model.pth")

# Load the best model
model.load_state_dict(torch.load("best_hdc_model.pth"))
print("Best model loaded!")

Epoch 1/50, Loss: 2.2698955551139326
Test Accuracy: 56.00%
Epoch 2/50, Loss: 2.1827484783587425
Test Accuracy: 64.15%
Epoch 3/50, Loss: 2.0895802232502367
Test Accuracy: 65.91%


KeyboardInterrupt: 

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from sklearn.metrics import accuracy_score

# Hyperparameters
DIMENSION = 10000  # High-dimensional space
PERTURBATION_SCALE = 0.0005  # Reduced perturbation
LEARNING_RATE = 0.1  # Gradual reinforcement rate

def random_hv():
    return np.random.choice([-1, 1], DIMENSION)

def structured_hv(value, min_val, max_val):
    # Structured encoding based on pixel intensity
    threshold = (value - min_val) / (max_val - min_val)
    return np.where(np.random.rand(DIMENSION) < threshold, 1, -1)

def bind(vec1, vec2):
    return vec1 * vec2

def bundle(vecs):
    return np.sign(np.sum(vecs, axis=0))

def similarity(vec1, vec2):
    return np.dot(vec1, vec2)

def adaptive_redistribute(vec, feedback):
    # Controlled redistribution based on feedback
    perturbation = np.random.choice([-1, 1], DIMENSION) * PERTURBATION_SCALE * max(0, (1 - feedback))
    return np.sign(vec + perturbation)

def reinforce(memory_vec, input_vec, correct):
    # Gradual reinforcement with learning rate
    adjustment = LEARNING_RATE * (input_vec if correct else -input_vec)
    return np.sign(memory_vec + adjustment)

# Load Fashion MNIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Flatten and normalize data
X_train = X_train.reshape(-1, 28*28) / 255.0
X_test = X_test.reshape(-1, 28*28) / 255.0

# Structured feature encoding
min_vals = X_train.min(axis=0)
max_vals = X_train.max(axis=0)
feature_hvs = [random_hv() for _ in range(X_train.shape[1])]
class_hvs = {label: random_hv() for label in np.unique(y_train)}

# Training phase
memory_vectors = {label: np.zeros(DIMENSION) for label in class_hvs}
for x, y in zip(X_train, y_train):
    encoded_features = [bind(feature_hvs[i], structured_hv(feature, min_vals[i], max_vals[i])) for i, feature in enumerate(x)]
    bundled_vector = bundle(encoded_features)
    feedback = similarity(bundled_vector, memory_vectors[y]) / DIMENSION
    bundled_vector = adaptive_redistribute(bundled_vector, feedback)
    memory_vectors[y] = reinforce(memory_vectors[y], bundled_vector, True)

# Normalize memory vectors
for label in memory_vectors:
    memory_vectors[label] = np.sign(memory_vectors[label])

# Testing phase
predictions = []
for x, true_label in zip(X_test, y_test):
    encoded_features = [bind(feature_hvs[i], structured_hv(feature, min_vals[i], max_vals[i])) for i, feature in enumerate(x)]
    bundled_vector = bundle(encoded_features)
    sims = {label: similarity(bundled_vector, mem_vec) for label, mem_vec in memory_vectors.items()}
    predicted_label = max(sims, key=sims.get)
    predictions.append(predicted_label)
    # Gradual reinforcement for correct predictions
    memory_vectors[true_label] = reinforce(memory_vectors[true_label], bundled_vector, predicted_label == true_label)

# Evaluate
accuracy = accuracy_score(y_test, predictions)
print(f"Model accuracy on Fashion MNIST with structured encoding and gradual reinforcement: {accuracy:.4f}")

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Model accuracy on Fashion MNIST with structured encoding and gradual reinforcement: 0.4140
